# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# COMPLETED: Import the necessary libs
# For example: 
import os
import json
from typing import List, Optional, TypedDict, Union
from dotenv import load_dotenv
from pydantic import BaseModel

import chromadb

from tavily import TavilyClient

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage, ErrorMessage
from lib.tooling import Tool, tool


In [3]:
# COMPLETED: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [4]:
# Simple debug logging
debug = True
print_debug = lambda x: print(f"DEBUG - {x}") if debug else None

### Tools

In [5]:
# initialize ChromeDB client
chroma_client = chromadb.PersistentClient(path="chromadb")

# test if data is already loaded
collection_name = "udaplay"

chrome_connection = chroma_client.get_collection(collection_name)
if chrome_connection.count() > 0:
    print(f"Collection '{collection_name}' already exists with {chrome_connection.count()} items.")
else:
    print(f"Collection '{collection_name}' does not exist or is empty. Please load data first.")


Collection 'udaplay' already exists with 15 items.


Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


In [6]:
class GameInfoResult(BaseModel):
    """Represents a result from querying the RAG system for."""
    content: str

class RAGEvalResult(BaseModel):
    """Represents the evaluation result of a RAG system."""
    useful: bool
    description: str

class WebSearchResult(BaseModel):
    """Represents a web search result."""
    title: str
    url: str
    content: str 
    score: float

#### Retrieve Game Tool

In [7]:
# COMPLETED: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

from typing import List

def retrieve_game(query: str) -> List[str]:
    """
    Search for game information in the knowledge base.
    """
    print_debug(f"Retrieving game information for query: {query}")
    res = chrome_connection.query(query_texts=[query], n_results=3,)
    return [doc for doc in res['documents'][0]]

@tool
def retrieve_game_tool(query: str) -> List[str]:
    """
    Tool to retrieve game information from the knowledge base.
    
    Args:
        query (str): A question about the game industry.

    Returns:
        List[str]: A list of game information strings, each containing:
            - Platform: like Game Boy, Playstation 5, Xbox 360...)
            - Name: Name of the Game
            - YearOfRelease: Year when that game was released for that platform
            - Description: Additional details about the game
    """
    return retrieve_game(query)


#### Evaluate Retrieval Tool

In [8]:
# COMPLETED: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

def evaluate_retrieval(question: str, retrieved_docs: List[str]) -> List[RAGEvalResult]:
    """
    Evaluate the usefulness of retrieved documents for a given question.
    """
    
    llm = LLM(model="gpt-4o-mini", temperature=0.0, tools=[], api_key=OPENAI_API_KEY)

    results: List[RAGEvalResult] = []

    evaluation_prompt = """
    You are an expert in evaluating the relevance of documents to user queries.
    Your task is to evaluate if the documents are enough to respond to the query. 

    Think step by step and provide an explanation of your evaluation result, including any specific issues or strengths of the documents.

    Provide your evaluation result as a valid JSON object:
    {
        "useful": true or false,
        "description": "a detailed explanation of your evaluation result, including any specific issues or strengths of the documents"
    }
    """
    print_debug(f"Evaluating retrieval for question: {question} with {len(retrieved_docs)} documents.")
    # Iterate over each retrieved document and evaluate it individually
    for doc in retrieved_docs:

        messages = [
            SystemMessage(
                content=evaluation_prompt
            ),
            UserMessage(
                content=(
                    f"Question: {question}\nRetrieved Document: {doc}"
                )
            )
        ]
        
        ai_response = llm.invoke(input=messages, response_format=RAGEvalResult)
        res = RAGEvalResult.model_validate_json(ai_response.content)
        results.append(res)

    print(f"Retrieval Evaluation Results: {results}")
    return results

@tool
def evaluate_retrieval_tool(question: str, retrieved_docs: List[str]) -> List[RAGEvalResult]:
    """
    Tool to evaluate the usefulness of retrieved documents for a given question.
    
    Args:
        question (str): The original question from the user.
        retrieved_docs (List[str]): The retrieved documents most similar to the user query in the Vector Database.

    Returns:
        List[RAGEvalResult]: A list of evaluation results, each containing:
            - useful: whether the documents are useful to answer the question
            - description: description about the evaluation result
    """
    return evaluate_retrieval(question, retrieved_docs)
                                                                              

#### Game Web Search Tool

In [9]:
# COMPLETED: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 
from datetime import datetime

def game_web_search(query: str) -> List[dict]:
    """
    Search for game-related information on the web.
    """
    print_debug(f"Performing web search for query: {query}")
    client = TavilyClient(api_key=TAVILY_API_KEY)
    search_depth = "advanced"
    search_result = client.search(
        query=query,
        max_results=2,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    print(f"Web search results for query '{query}':")

    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results


@tool
def game_web_search_tool(query: str) -> List[WebSearchResult]:
    """
    Tool to search for game-related information on the web.
    
    Args:
        query (str): A question about the game industry.

    Returns:
        List[WebSearchResult]: A list of web search results, each containing:
            - title: Title of the search result
            - url: URL of the search result
            - content: Content of the search result
            - score: Relevance score of the search result
    """
    results = game_web_search(query)
    return [WebSearchResult(**result) for result in results['results']] 

### Agent

In [10]:
# COMPLETED: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
tools = [retrieve_game_tool, evaluate_retrieval_tool, game_web_search_tool]
agent_instructions = """
Your are a helpful assistant that can answer questions about the game industry. 

Tools:
- retrieve_game: A tool to retrieve information about a specific game.
- evaluate_retrieval: A tool to evaluate the relevance of retrieved documents.
- game_web_search: A tool to search the web for additional information about games.

Instructions for using tools:
1. Use the `retrieve_game` tool to find information about a specific game.
2. After retrieving documents, use the `evaluate_retrieval` tool to assess their relevance to the user's question.
3. If the retrieved documents are not sufficient or if zero games were retrieved, use the `game_web_search` tool to search the web for more information.
4. Always provide a clear and concise answer to the user's question

"""

agent = Agent(model_name="gpt-4o-mini", instructions=agent_instructions, tools=tools)

In [11]:
import uuid
session_id = uuid.uuid4().hex
print(f"Session ID: {session_id}")

Session ID: a94f5bfd5b874dfa802583170176be4b


In [12]:
# COMPLETED: Invoke your agent
q1 =" When Pokémon Gold and Silver was released?"
q2="Which one was the first 3D platformer Mario game?"
q3 = "Was Mortal Kombat X realeased for Playstation 5?"

In [13]:
run1 = agent.invoke(query=q1, session_id=session_id)
print(f"Run 1: {run1.get_final_state()['messages'][-1].content}")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
DEBUG - Retrieving game information for query: Pokémon Gold and Silver release date
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
DEBUG - Evaluating retrieval for question: When Pokémon Gold and Silver was released? with 1 documents.
Retrieval Evaluation Results: [RAGEvalResult(useful=True, description="The retrieved document provides the release year of Pokémon Gold and Silver as 1999, which directly answers the user's query about when the games were released. Additionally, it mentions that these games are for the Game Boy Color and introduces new regions, Pokémon, and gameplay mechanics, which adds context and relevance to the information. There are no significant issues with the document, as it is concise and informative.")]
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[Sta

In [14]:
run2 = agent.invoke(query=q2, session_id=session_id)
print("Query: ", q2)
print(f"Run 2: {run2.get_final_state()['messages'][-1].content}")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
DEBUG - Retrieving game information for query: first 3D platformer Mario game
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
DEBUG - Evaluating retrieval for question: Which one was the first 3D platformer Mario game? with 1 documents.
Retrieval Evaluation Results: [RAGEvalResult(useful=True, description="The retrieved document provides a clear and direct answer to the query regarding the first 3D platformer Mario game. It specifies 'Super Mario 64' as the title and includes the release year (1996), which is relevant information. Additionally, it mentions the game's significance in setting new standards for the genre, which adds context to its importance. There are no issues with the document; it is concise and informative, effectively addressing the user's question.")]
[StateMachine] Executing step: tool_executor
[StateM

In [15]:
# Inspecting the messages of run 2
for m in run2.get_final_state()['messages']:

    if isinstance(m, ToolMessage):
        print(f"Tool: {m.content}")
    elif isinstance(m, AIMessage):
        print(f"AI: {m.content}")
    else:
        print(f"Message: {m.content}")

Message: 
Your are a helpful assistant that can answer questions about the game industry. 

Tools:
- retrieve_game: A tool to retrieve information about a specific game.
- evaluate_retrieval: A tool to evaluate the relevance of retrieved documents.
- game_web_search: A tool to search the web for additional information about games.

Instructions for using tools:
1. Use the `retrieve_game` tool to find information about a specific game.
2. After retrieving documents, use the `evaluate_retrieval` tool to assess their relevance to the user's question.
3. If the retrieved documents are not sufficient or if zero games were retrieved, use the `game_web_search` tool to search the web for more information.
4. Always provide a clear and concise answer to the user's question


Message:  When Pokémon Gold and Silver was released?
AI: None
Tool: "['[Game Boy Color] Pok\u00e9mon Gold and Silver (1999) - Second-generation Pok\u00e9mon games introducing new regions, Pok\u00e9mon, and gameplay mechanic

In [16]:
run3 = agent.invoke(query=q3, session_id=session_id)
print("Query: ", q3)
print(f"Run 3: {run3.get_final_state()['messages'][-1].content}")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
DEBUG - Retrieving game information for query: Mortal Kombat X release platforms
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
DEBUG - Performing web search for query: Mortal Kombat X release platforms including Playstation 5
Web search results for query 'Mortal Kombat X release platforms including Playstation 5':
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Query:  Was Mortal Kombat X realeased for Playstation 5?
Run 3: Mortal Kombat X was not originally released for the PlayStation 5; it was released for the PlayStation 4 and Xbox One on April 14, 2015. However, it is backwards compatible with the PlayStation 5, meaning you can play the PlayStation 4 version on the PS5.


In [17]:
# Inspecting the messages of run 3
# This should show the history of the conversation
for m in run3.get_final_state()['messages']:

    if isinstance(m, ToolMessage):
        print(f"Tool: {m.content}")
    elif isinstance(m, AIMessage):
        print(f"AI: {m.content}")
    else:
        print(f"Message: {m.content}")

Message: 
Your are a helpful assistant that can answer questions about the game industry. 

Tools:
- retrieve_game: A tool to retrieve information about a specific game.
- evaluate_retrieval: A tool to evaluate the relevance of retrieved documents.
- game_web_search: A tool to search the web for additional information about games.

Instructions for using tools:
1. Use the `retrieve_game` tool to find information about a specific game.
2. After retrieving documents, use the `evaluate_retrieval` tool to assess their relevance to the user's question.
3. If the retrieved documents are not sufficient or if zero games were retrieved, use the `game_web_search` tool to search the web for more information.
4. Always provide a clear and concise answer to the user's question


Message:  When Pokémon Gold and Silver was released?
AI: None
Tool: "['[Game Boy Color] Pok\u00e9mon Gold and Silver (1999) - Second-generation Pok\u00e9mon games introducing new regions, Pok\u00e9mon, and gameplay mechanic

In [18]:
# run a query that will use the session memory: this should state that Mortal Kombat X was released for PlayStation
run4 = agent.invoke(query="Which games that I queried so far, were released on 'PlayStation'?", session_id=session_id)
print(f"Run 4: {run4.get_final_state()['messages'][-1].content}")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Run 4: From the games you queried so far, the following were released on PlayStation:

1. **Pokémon Gold and Silver** - Released for Game Boy Color, not on PlayStation.
2. **Super Mario 64** - Released for Nintendo 64, not on PlayStation.
3. **Mortal Kombat X** - Released for PlayStation 4, and is backwards compatible with PlayStation 5.

So, the only game released on PlayStation among those you queried is **Mortal Kombat X**.


In [19]:
# run a query in a new session to show that the session memory is not used: this should return a generic answer without reference to the games queried in the previous session
new_session_id = uuid.uuid4().hex
print(f"New Session ID: {new_session_id}")
run5 = agent.invoke(query="Which games that I queried so far, were released on 'PlayStation'?", session_id=new_session_id)
print(f"Run 5: {run5.get_final_state()['messages'][-1].content}")

New Session ID: a6a8a9bd69234f0797e4a32c922fc027
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
DEBUG - Retrieving game information for query: games released on PlayStation
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
DEBUG - Evaluating retrieval for question: Which games that I queried so far, were released on 'PlayStation'? with 3 documents.
Retrieval Evaluation Results: [RAGEvalResult(useful=True, description="The retrieved document provides relevant information regarding a specific game, Gran Turismo 5, which was released on the PlayStation 3 in 2010. It directly answers the query about games released on 'PlayStation' by identifying a specific title and its platform. The document could be improved by including additional titles or a broader context about other PlayStation games, but it is sufficient to respond to the query regarding at least one game."), RAGEval

### (Optional) Advanced

- TODO: Update your agent with long-term memory
- TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes

#### Agent with Long-Term Memory

The agent is extended to extract the platform information and game genre from user queries and store this information as a preference for the user's profile. The memory can be used for queries like: _"Show me latest games that I might like?"_. This version doesn't capture the profile information from the queries but manages them separately. This is only done to keep thing simple. 

User profile:
- platforms list of string representing the game platforms the user has queried information for.
- genres: list of strings representing various game genres the user has queried information for.

The implementation used the LongTermMemory class provided by the course. An alternative would be to use a SQL database since the data is rather well structured.


In [20]:
from lib.vector_db import VectorStoreManager
from lib.memory import LongTermMemory, MemoryFragment

In [21]:
db = VectorStoreManager(OPENAI_API_KEY)
vector_store = db.get_or_create_store("user_profiles")
ltm = LongTermMemory(db)


In [22]:
def build_memory_registration_tool(ltm:LongTermMemory, owner:str, namespace:str):
    """
    Create a tool for agents to register new memories.
    
    This factory function creates a tool that allows AI agents to store new
    information about users in the long-term memory system. The tool is
    pre-configured with specific owner and namespace parameters.
    
    Args:
        ltm (LongTermMemory): The memory system instance to use
        owner (str): User identifier for memory ownership
        namespace (str): Namespace for organizing memories
        
    Returns:
        Tool: A configured tool for memory registration
    """
    def _register(content:str):
        print_debug(f"Registering content: {content} for owner: {owner} in namespace: {namespace}")
        ltm.register(
            MemoryFragment(
                content=content, 
                owner=owner,
                namespace=namespace
            )
        )
        return "Saved new memory"

    return Tool(
        func=_register, 
        name="register_profile_info", 
        description=(
            "Register a new information or preference about the user, " 
            "so it can be useful later as context.\n"
            "Args:\n"
            "    content: The information to save"
        )
    )


def build_memory_search_tool(ltm:LongTermMemory, owner:str, namespace:str):
    """
    Create a tool for agents to search existing memories.
    
    This factory function creates a tool that allows AI agents to retrieve
    relevant memories from the long-term memory system based on semantic
    similarity to a search query.
    
    Args:
        ltm (LongTermMemory): The memory system instance to use
        owner (str): User identifier for memory ownership
        namespace (str): Namespace to search within
        
    Returns:
        Tool: A configured tool for memory search
    """
    def _search(content:str):
        print_debug(f"Searching for content: {content} in namespace: {namespace} for owner: {owner}")
        result = ltm.search(
            query_text=content,
            owner=owner,
            namespace=namespace,
            limit=3,
        )
        return str(tuple(zip(result.fragments, result.metadata['distances'])))

    return Tool(
        func=_search, 
        name="search_profile_info", 
        description=(
            "Search for a stored information or preference about the user, " 
            "so it's useful as a context.\n"
            "Args:\n"
            "    content: The information to look for"
        )
    )


In [23]:
user = "demo_user"

# insert some sample user preferences into the long-term memory
agent_preferences = {
    "favorite_genres": ["Action", "Adventure", "RPG"],
    "preferred_platforms": ["PlayStation"]
}

ltm.register(
    MemoryFragment(
        content=json.dumps(agent_preferences),
        owner=user,
        namespace="profiles"
    )
)

In [24]:


agent_instructions = """
Your are a helpful assistant that can answer questions about the game industry. 

Tools:
- retrieve_game: A tool to retrieve information about a specific game.
- evaluate_retrieval: A tool to evaluate the relevance of retrieved documents.
- game_web_search: A tool to search the web for additional information about games.
- search_profile_info: A tool to search for existing memories or preferences about the user.

Instructions for using tools:
- Use the `search_profile_info` to find any preference information about the user, such as favorite genres or platforms.
- Use the `retrieve_game` tool to find information about a specific game.
- After retrieving documents, use the `evaluate_retrieval` tool to assess their relevance to the user's question.
- If the retrieved documents are not sufficient or if zero games were retrieved, use the `game_web_search` tool to search the web for more information.
- Always provide a clear and concise answer to the user's question
- Use the `search_profile_info` tool to retrieve stored information about the user, which can help tailor responses to their interests and preferences.

Instructions for managing user profile information:
- When a user asks about their preferences or interests, use the `search_profile_info` tool to get relevant information from their preferences
- Extract preferences such as game platforms and genres from user queries and store them using the `register_profile_info` tool.
- Before answering questions about the user, check if relevant preferences are already stored using the `search_profile_info` tool.
"""

tools = tools + [
    build_memory_registration_tool(ltm, owner=user, namespace="profiles"),
    build_memory_search_tool(ltm, owner=user, namespace="profiles")
]

agent_with_memory = Agent(model_name="gpt-4o-mini", instructions=agent_instructions, tools=tools)

In [25]:
run_lt_1 = agent_with_memory.invoke(query="When was the first 3D platformer Mario game released?")
print(f"Run 1: {run_lt_1.get_final_state()['messages'][-1].content}")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
DEBUG - Retrieving game information for query: first 3D platformer Mario game release date
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
DEBUG - Evaluating retrieval for question: When was the first 3D platformer Mario game released? with 3 documents.
Retrieval Evaluation Results: [RAGEvalResult(useful=True, description="The retrieved document provides a clear and direct answer to the query by stating that 'Super Mario 64' was released in 1996, which is indeed the first 3D platformer game featuring Mario. It also highlights the significance of the game in setting new standards for the genre, which adds context to its importance. There are no issues with the document as it directly addresses the question asked."), RAGEvalResult(useful=False, description="The retrieved document mentions 'Super Mario World' which is a class

In [26]:
run_lt_2 = agent_with_memory.invoke(query="What is the most recent game that I should play?")
print(f"Run 2: {run_lt_2.get_final_state()['messages'][-1].content}")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
DEBUG - Searching for content: latest games in namespace: profiles for owner: demo_user
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
DEBUG - Performing web search for query: latest games 2023 PlayStation Action Adventure RPG
Web search results for query 'latest games 2023 PlayStation Action Adventure RPG':
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Run 2: Based on your interests in Action, Adventure, and RPG games on PlayStation, here are some of the most recent games you should consider playing:

1. **Baldur's Gate 3** - This RPG has been highly praised and is considered one of the best games of 2023. It offers a deep narrative and complex characters.

   [Read more about Baldur's Gate 3](https://www.ign.com/articles/best-rpg-gam

#### Agent Implementation with State Machine

The following code implements the optional assignment to use a state machine for implementing the process for answering user questions related to video games. A state machine is well suited for this task since the process is defined in a sequential number of steps with only conditional paths (for web search if no relevant documents are found in the knowledge base).

In [27]:
from pydantic import Field

from lib.tooling import ToolCall
from lib.state_machine import StateMachine, Step, EntryPoint, Termination


In [28]:


class AgentState(TypedDict):
    user_query: str  
    instructions: str
    messages: List[dict]
    documents: Optional[List[str]]  = Field(default_factory=list)


def prepare_messages_step(state: AgentState) -> AgentState:
    """
    Prepare messages for the agent's state.
    
    This function processes the current state and prepares the messages for the agent.
    It ensures that the messages are in the correct format and ready for further processing.
    
    Args:
        state (AgentState): The current state of the agent.
        
    Returns:
        AgentState: The updated state with prepared messages.
    """
    # Here you can implement any logic to prepare messages
    # For example, you might want to format or filter messages
    messages = [
        SystemMessage(content=state['instructions']),
        UserMessage(content=state['user_query'])
    ]
    return {
        "messages": messages,
    }

def generate_result(state: AgentState) -> AgentState:
    """
    Generate the final result based on the current state.
    
    This function processes the messages and generates the final result for the agent.
    
    Args:
        state (AgentState): The current state of the agent.
        
    Returns:
        AgentState: The updated state with the final result.
    """

    # initialize the LLM

    messages = state['messages']

    document_context = "\n".join(state['documents']) if state['documents'] else "No relevant documents found."

    messages.append(
        AIMessage(content=f"Context for answering the user query:\n {document_context}")
    )

    llm = LLM(model="gpt-4o-mini", temperature=0.0, tools=[], api_key=OPENAI_API_KEY)
    ai_response = llm.invoke(state['messages'])

    return {
        "messages": state["messages"] + [ai_response],
    }   


def error_handler(state: AgentState) -> AgentState:
    """
    Handle errors that occur during the agent's execution.
    
    This function processes the current state and handles any non-recoverable errors that may have occurred. 
    
    Args:
        state (AgentState): The current state of the agent.
        
    Returns:
        AgentState: The updated state after handling the error.
    """
    print("An error occurred during the agent's execution. The agent will now terminate.")
    return {
        "messages": state["messages"] + [ErrorMessage(content="An error occurred during the agent's execution.")],
        "current_tool_calls": None
    }


In [29]:
def retrieve_game_state(state: AgentState) -> Agent:
    """
    Search for game information in the knowledge base.
    """
    docs = retrieve_game(state['user_query'])
    
    return {
        "messages": state["messages"],
        "documents": docs
    }

def evaluate_retrieval_state(state: AgentState) -> AgentState:
    """
    Evaluate the relevance of retrieved documents to the user's query.
    
    This function processes the current state and evaluates the retrieved documents using the `evaluate_retrieval` tool.
    
    Args:
        state (AgentState): The current state of the agent.
        
    Returns:
        AgentState: The updated state with evaluation results.
    """
    print("Evaluating retrieval results...")
    evaluation_results = evaluate_retrieval(state['user_query'], state['documents'])
    print_debug("Evaluation results:")
    for eval in evaluation_results:
        print_debug(f"Useful: {eval.useful}, Description: {eval.description}")


    evaluated_docs = [doc for eval, doc in zip(evaluation_results, state['documents']) if eval.useful]
    print_debug(f"Relevant documents after evaluation: {len(evaluated_docs)} of {len(state['documents'])} retrieved documents.")

    for evaluated_doc in evaluated_docs:
        print_debug(f"Evaluated Document: {evaluated_doc}")

    return {
        "messages": state["messages"],
        "documents": evaluated_docs
    }

def web_search_state(state: AgentState) -> AgentState:
    """
    Search the web for additional information about the game.
    
    This function processes the current state and performs a web search using the `game_web_search` tool.
    
    Args:
        state (AgentState): The current state of the agent.
        
    Returns:
        AgentState: The updated state with web search results.
    """
    web_results = game_web_search(state['user_query'])
    
    return {
        "messages": state["messages"],
        "documents": [result["content"] for result in web_results["results"]]
    }


In [30]:
workflow = StateMachine[AgentState](AgentState)

# create steps
entry_step = EntryPoint[AgentState]()
message_prep_step = Step[AgentState]("message_prep", prepare_messages_step)
tool_retrieve_game_step = Step[AgentState]("tool_retrieve_game", retrieve_game_state)
tool_evaluate_retrieval_step = Step[AgentState]("tool_evaluate_retrieval", evaluate_retrieval_state)
tool_game_web_search_step = Step[AgentState]("tool_game_web_search", web_search_state)
generate_result_step = Step[AgentState]("generate_result", generate_result)
error_handler_step = Step[AgentState]("error_handler", error_handler)
termination_step = Termination[AgentState]()

# add steps to workflow
workflow.add_steps(
    [
        entry_step,
        message_prep_step,
        tool_retrieve_game_step,
        tool_evaluate_retrieval_step,
        tool_game_web_search_step,
        generate_result_step,
        error_handler_step,
        termination_step
    ]
)

# define condition on result evaluation  
def check_relevant_docs_condition(state: AgentState) -> Union[Step[AgentState], str]:
    """
    Check if the evaluation of the retrieved documents is sufficient.
    
    This function checks the last AI message in the state to determine if the evaluation
    indicates that the retrieved documents are useful.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        bool: True if the evaluation is sufficient, False otherwise.
    """
    relevant_docs = state["documents"]

    if not relevant_docs:
        print("No relevant documents found. Proceeding to web search.")
        return tool_game_web_search_step
    else:
        print(f"Found {len(relevant_docs)} relevant documents. Proceeding to generate result.")
        return generate_result_step


# define transitions
workflow.connect(entry_step, message_prep_step)
workflow.connect(message_prep_step, tool_retrieve_game_step)
workflow.connect(tool_retrieve_game_step, tool_evaluate_retrieval_step)
workflow.connect(tool_game_web_search_step, generate_result_step)
workflow.connect(generate_result_step, termination_step)

# condition to call web-search only if the retrieval did not return any relevant documents
workflow.connect(source=tool_evaluate_retrieval_step, targets=[generate_result_step,tool_game_web_search_step], condition=check_relevant_docs_condition)

In [31]:
#user_query = "When Pokémon Gold and Silver was released?"
user_query = "Was Mortal Kombat X realeased for Playstation 5?"


In [32]:
# define the initial state
initial_state: AgentState = {
    "user_query": user_query,
    "instructions": agent_instructions,
    "messages": [],
}

In [33]:
run_object = workflow.run(initial_state)
run_object.get_final_state()["messages"]

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
DEBUG - Retrieving game information for query: Was Mortal Kombat X realeased for Playstation 5?
[StateMachine] Executing step: tool_retrieve_game
Evaluating retrieval results...
DEBUG - Evaluating retrieval for question: Was Mortal Kombat X realeased for Playstation 5? with 3 documents.
Retrieval Evaluation Results: [RAGEvalResult(useful=False, description="The retrieved document does not provide any relevant information regarding the release of Mortal Kombat X for PlayStation 5. Instead, it discusses Marvel's Spider-Man 2, which is unrelated to the query. There is no mention of Mortal Kombat X or its availability on any platform, let alone the PlayStation 5. Therefore, the document is not useful for answering the user's question."), RAGEvalResult(useful=False, description="The retrieved document is completely irrelevant to the query about the release of Mortal Kombat X for PlayStation 5. It discusses Gran T

[SystemMessage(role='system', content="\nYour are a helpful assistant that can answer questions about the game industry. \n\nTools:\n- retrieve_game: A tool to retrieve information about a specific game.\n- evaluate_retrieval: A tool to evaluate the relevance of retrieved documents.\n- game_web_search: A tool to search the web for additional information about games.\n- search_profile_info: A tool to search for existing memories or preferences about the user.\n\nInstructions for using tools:\n- Use the `search_profile_info` to find any preference information about the user, such as favorite genres or platforms.\n- Use the `retrieve_game` tool to find information about a specific game.\n- After retrieving documents, use the `evaluate_retrieval` tool to assess their relevance to the user's question.\n- If the retrieved documents are not sufficient or if zero games were retrieved, use the `game_web_search` tool to search the web for more information.\n- Always provide a clear and concise a